In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import torch

from wittgenstein.irep import IREP
from wittgenstein.ripper import RIPPER
from wittgenstein.interpret import interpret_model, score_fidelity

In [2]:
# Load data and transform target to binary classification 

inpath = ''
df = pd.read_csv(inpath + "wine.csv")
df['Class'] = df['Class'].map(lambda x: True if x==1 else False)
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,True,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,True,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,True,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,True,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,True,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
# Train-test-split

X, y = df.drop('Class', axis=1), df['Class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

In [4]:
# Create a wittgenstein classifier for (global surrogate) interpretation.

interpreter = RIPPER(random_state=42)

In [5]:
# Fit and interpret SVM.
# interpret_model fits the interpreter to the model.

svc = SVC(kernel='rbf', random_state=42)
svc.fit(X_train, y_train);

interpret_model(model=svc, X=X_train, interpreter=interpreter).out_pretty()


[[Proline=>1227.0] V
[Proline=1048.0-1227.0] V
[Proline=880.0-1048.0]]


In [6]:
# Score how well the interpreter's predictions match the underlying model's

score_fidelity(
    X_test,
    interpreter,
    model=svc,
    score_function=[precision_score, recall_score, f1_score])

[1.0, 1.0, 1.0]

In [7]:
# Interpret Random Forest

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

interpret_model(model=rf, X=X_train, interpreter=interpreter).out_pretty()


[[Proline=>1227.0] V
[Proline=1048.0-1227.0] V
[Proline=880.0-1048.0 ^ Ash=2.0-2.19] V
[Proline=880.0-1048.0 ^ Malicacid=1.76-1.9] V
[Colorintensity=5.75-6.78]]


In [8]:
score_fidelity(
    X_test,
    interpreter,
    model=rf,
    score_function=[precision_score, recall_score, f1_score])

[1.0, 0.7333333333333333, 0.846153846153846]

In [19]:
# Interpret Tensorflow/Keras model

np.random.seed(0)

mlp = Sequential()
mlp.add(Dense(60, input_dim=13, activation='relu'))
mlp.add(Dense(30, activation='relu'))
mlp.add(Dense(1, activation='sigmoid'))
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=10,
)

interpret_model(model=mlp, X=X_train, interpreter=interpreter).out_pretty()


Epoch 1/10
133/133 [==============================] - 1s 2ms/step - loss: 4.2813 - accuracy: 0.7293
Epoch 2/10
133/133 [==============================] - 0s 2ms/step - loss: 1.8663 - accuracy: 0.7744
Epoch 3/10
133/133 [==============================] - 0s 1ms/step - loss: 3.7320 - accuracy: 0.7143
Epoch 4/10
133/133 [==============================] - 0s 2ms/step - loss: 1.7408 - accuracy: 0.8271
Epoch 5/10
133/133 [==============================] - 0s 2ms/step - loss: 1.4441 - accuracy: 0.8271
Epoch 6/10
133/133 [==============================] - 0s 2ms/step - loss: 1.7489 - accuracy: 0.8045
Epoch 7/10
133/133 [==============================] - 0s 1ms/step - loss: 2.2349 - accuracy: 0.8195
Epoch 8/10
133/133 [==============================] - 0s 2ms/step - loss: 1.3922 - accuracy: 0.8421
Epoch 9/10
133/133 [==============================] - 0s 3ms/step - loss: 1.1142 - accuracy: 0.8872
Epoch 10/10
133/133 [==============================] - 0s 2ms/step - loss: 1.4097 - accuracy: 0.8647

In [10]:
score_fidelity(
    X_test,
    interpreter,
    model=mlp,
    score_function=[precision_score, recall_score, f1_score])

[1.0, 0.7916666666666666, 0.8837209302325582]

In [31]:
# Since interpret_model fits a wittgenstein classifier,
# we can do with it all the normal things a wittgenstein classifier can do.
# Here, we'll use it to approximate what in the data the tensorflow model
# is picking up on when it makes specific positive predictions.

preds = (mlp.predict(X_test.tail()) > .5).flatten()
_, interpretation = interpreter.predict(X_test.tail(), give_reasons=True)

print(f'tf preds: {preds}\n')
print('interpretion:')
interpretation


tf preds: [ True False False  True False]

interpretion:


[[<Rule [Proline=880.0-1048.0]>],
 [],
 [],
 [<Rule [Alcalinityofash=16.8-17.72]>],
 []]